In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel("/content/Fraud data FY 2023-24 for B&CC.xlsx")

In [ ]:
pip install pandas plotly


In [ ]:
import pandas as pd
import plotly.express as px

# Load your dataset
  # replace with actual path

# Convert date columns to datetime
df['POLICYRISKCOMMENCEMENTDATE'] = pd.to_datetime(df['POLICYRISKCOMMENCEMENTDATE'], errors='coerce')
df['Date of Death'] = pd.to_datetime(df['Date of Death'], errors='coerce')
df['INTIMATIONDATE'] = pd.to_datetime(df['INTIMATIONDATE'], errors='coerce')

# Create date difference columns
df['Policy_to_Death_Days'] = (df['Date of Death'] - df['POLICYRISKCOMMENCEMENTDATE']).dt.days
df['Death_to_Intimation_Days'] = (df['INTIMATIONDATE'] - df['Date of Death']).dt.days


In [ ]:
df.columns

Index(['Dummy Policy No', 'ASSURED_AGE', 'NOMINEE_RELATION', 'OCCUPATION',
       'POLICY SUMASSURED', 'Premium', 'PREMIUMPAYMENTMODE', 'Annual Income',
       'HOLDERMARITALSTATUS', 'INDIV_REQUIREMENTFLAG', 'Policy Term',
       'Policy Payment Term', 'CORRESPONDENCECITY', 'CORRESPONDENCESTATE',
       'CORRESPONDENCEPOSTCODE', 'Product Type', 'CHANNEL', 'Bank code',
       'POLICYRISKCOMMENCEMENTDATE', 'Date of Death', 'INTIMATIONDATE',
       'STATUS', 'SUB_STATUS', 'Fraud Category', 'Policy_to_Death_Days',
       'Death_to_Intimation_Days'],
      dtype='object')

In [ ]:
fig = px.treemap(df,
                 path=['CORRESPONDENCESTATE', 'CORRESPONDENCECITY'],
                 values=[1]*len(df),
                 title="State-City-wise Fraud Count")
fig.show()


In [ ]:
fig = px.bar(df,
             x='CHANNEL',
             title='Fraud Count by Channel')
fig.show()


In [ ]:
fig = px.histogram(df,
                   x='Policy_to_Death_Days',
                   nbins=30,
                   title='Days Between Policy Start and Death')
fig.show()


In [ ]:
fig = px.histogram(df,
                   x='Death_to_Intimation_Days',
                   nbins=30,
                   title='Days Between Death and Intimation')
fig.show()


In [24]:
import pandas as pd
import plotly.express as px
fig = px.bar(df,
             x='CORRESPONDENCESTATE',
             color='CHANNEL',
             title='State-wise Distribution of CHANNELs',
             category_orders={'CORRESPONDENCESTATE': df['CORRESPONDENCESTATE'].unique()},
             labels={'CORRESPONDENCESTATE': 'State', 'CHANNEL': 'Channel'},
             barmode='group')
fig.update_layout(
    hovermode='x unified',
    showlegend=True,
    title={'x': 0.5}
)

fig.show()


In [28]:
fraud_data = df[df['Fraud Category'].notna()]
fraud_counts = fraud_data.groupby(['CORRESPONDENCEPOSTCODE', 'CORRESPONDENCECITY']).size().reset_index(name='Fraud Count')
top_frauds = fraud_counts.sort_values(by='Fraud Count', ascending=False).head(10)
top_frauds['Postcode + City'] = top_frauds['CORRESPONDENCEPOSTCODE'] + ' - ' + top_frauds['CORRESPONDENCECITY']
fig = px.bar(top_frauds[1:],
             x='Postcode + City',  # Use the combined text of postcode and city
             y='Fraud Count',
             color='CORRESPONDENCECITY',
             hover_data=['Postcode + City', 'Fraud Count'],
             title='Top 10 Postcodes with Highest Number of Frauds',
             labels={'Postcode + City': 'Postcode - City', 'Fraud Count': 'Number of Frauds'},
             color_continuous_scale='Viridis',  # You can choose another color scale
             text='Fraud Count')
fig.update_layout(
    xaxis_title='Postcode - City',
    yaxis_title='Fraud Count',
    hovermode='closest',
    showlegend=False,  # Hide legend to avoid clutter
    title={'x': 0.5},  # Center the title
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent background
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
)

fig.show()


# plots improved 2.0



In [29]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio


In [30]:
pio.templates.default = "plotly_white"

# Load your dataset (replace with actual path)
df = pd.read_excel("Fraud data FY 2023-24 for B&CC.xlsx")

# Data preprocessing
# Convert date columns to datetime
df['POLICYRISKCOMMENCEMENTDATE'] = pd.to_datetime(df['POLICYRISKCOMMENCEMENTDATE'], errors='coerce')
df['Date of Death'] = pd.to_datetime(df['Date of Death'], errors='coerce')
df['INTIMATIONDATE'] = pd.to_datetime(df['INTIMATIONDATE'], errors='coerce')

# Create date difference columns
df['Policy_to_Death_Days'] = (df['Date of Death'] - df['POLICYRISKCOMMENCEMENTDATE']).dt.days
df['Death_to_Intimation_Days'] = (df['INTIMATIONDATE'] - df['Date of Death']).dt.days

# Add month and year columns for time series analysis
df['Month_Year'] = df['INTIMATIONDATE'].dt.to_period('M').astype(str)
df['Year'] = df['INTIMATIONDATE'].dt.year


In [32]:
# 2. Multi-metric Channel Analysis - Bar Chart with Custom Hover
channel_summary = df.groupby('CHANNEL').agg(
    Count=('CHANNEL', 'size'),
    Avg_Policy_to_Death=('Policy_to_Death_Days', 'mean'),
    Avg_Death_to_Intimation=('Death_to_Intimation_Days', 'mean')
).reset_index()

fig2 = px.bar(
    channel_summary,
    x='CHANNEL',
    y='Count',
    color='Avg_Policy_to_Death',
    color_continuous_scale='Plasma',
    hover_data=['Avg_Policy_to_Death', 'Avg_Death_to_Intimation'],
    labels={'Count': 'Number of Fraud Cases', 'CHANNEL': 'Channel'},
    title="<b>Fraud Analysis by Channel</b><br><i>Color indicates average days from policy start to death</i>"
)
fig2.update_traces(marker_line_width=1.5, marker_line_color='white')
fig2.update_layout(
    height=500,
    hoverlabel=dict(bgcolor="white", font_size=12),
    coloraxis_colorbar=dict(title="Avg Days<br>Policy to Death")
)
fig2.show()

In [34]:
fig4 = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "Days Between Policy Start and Death",
        "Days Between Death and Intimation"
    )
)

# First histogram
fig4.add_trace(
    go.Histogram(
        x=df['Policy_to_Death_Days'],
        nbinsx=30,
        marker_color='rgba(73, 133, 186, 0.7)',
        marker_line_color='rgba(73, 133, 186, 1)',
        marker_line_width=1,
        name="Policy to Death"
    ),
    row=1, col=1
)

# Second histogram
fig4.add_trace(
    go.Histogram(
        x=df['Death_to_Intimation_Days'],
        nbinsx=30,
        marker_color='rgba(214, 39, 40, 0.7)',
        marker_line_color='rgba(214, 39, 40, 1)',
        marker_line_width=1,
        name="Death to Intimation"
    ),
    row=1, col=2
)

fig4.update_layout(
    title_text="<b>Distribution Analysis of Time Intervals</b>",
    showlegend=False,
    height=500,
    bargap=0.1,
    bargroupgap=0.2,
    hovermode="x unified"
)

# Add vertical lines for average values
fig4.add_vline(
    x=df['Policy_to_Death_Days'].median(),
    line_dash="dash", line_color="black",
    annotation_text=f"Median: {df['Policy_to_Death_Days'].median():.0f} days",
    row=1, col=1
)

fig4.add_vline(
    x=df['Death_to_Intimation_Days'].median(),
    line_dash="dash", line_color="black",
    annotation_text=f"Median: {df['Death_to_Intimation_Days'].median():.0f} days",
    row=1, col=2
)

fig4.show()

In [36]:
# 5. Top Fraud Hotspots - Advanced Bar Chart
fraud_data = df[df['Fraud Category'].notna()]
fraud_counts = fraud_data.groupby(['CORRESPONDENCEPOSTCODE', 'CORRESPONDENCECITY', 'CORRESPONDENCESTATE']).size().reset_index(name='Fraud_Count')
top_frauds = fraud_counts.sort_values(by='Fraud_Count', ascending=False).head(10)
top_frauds['Location'] = top_frauds['CORRESPONDENCECITY'] + ' (' + top_frauds['CORRESPONDENCEPOSTCODE'].astype(str) + ')'

fig5 = px.bar(
    top_frauds[1:],
    x='Location',
    y='Fraud_Count',
    color='CORRESPONDENCESTATE',
    text='Fraud_Count',
    title="<b>Top 10 Fraud Hotspots</b>",
    labels={'Location': 'Location (City & Postcode)', 'Fraud_Count': 'Number of Fraud Cases'},
    hover_data=['CORRESPONDENCESTATE', 'CORRESPONDENCEPOSTCODE', 'CORRESPONDENCECITY', 'Fraud_Count']
)

fig5.update_traces(
    textposition='outside',
    textfont=dict(size=12),
    marker_line_width=1.5,
    marker_line_color='white'
)

fig5.update_layout(
    height=500,
    xaxis_tickangle=-45,
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    yaxis_title="Number of Fraud Cases",
    xaxis_title="",
    legend_title="State",
    hovermode="closest"
)

fig5.show()

In [40]:
# 6. Relationship Analysis - Scatter Plot with Trend Line (Fixed)
# First, filter out rows with NaN values in the key columns
scatter_df = df.dropna(subset=['Policy_to_Death_Days', 'Death_to_Intimation_Days'])

# Create a fixed size value that won't have NaN issues
scatter_df['size_value'] = scatter_df['Policy_to_Death_Days'].clip(lower=1, upper=100)

fig6 = px.scatter(
    scatter_df,  # Use the filtered dataframe
    x='Policy_to_Death_Days',
    y='Death_to_Intimation_Days',
    color='CHANNEL',
    size='size_value',  # Use the cleaned size column
    size_max=15,
    opacity=0.7,
    hover_data=['CORRESPONDENCESTATE', 'CORRESPONDENCECITY'],
    title="<b>Relationship Between Policy-to-Death and Death-to-Intimation Intervals</b>",
    trendline="ols",
    trendline_color_override="black"
)

fig6.update_layout(
    height=600,
    xaxis_title="Days Between Policy Start and Death",
    yaxis_title="Days Between Death and Intimation",
    legend_title="Channel",
    hovermode="closest"
)

fig6.show()

<ipython-input-40-f77d8a40a074>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
# 7. Channel Distribution by State - Interactive Grouped Bar Chart
state_channel = df.groupby(['CORRESPONDENCESTATE', 'CHANNEL']).size().reset_index(name='Count')
pivot_data = state_channel.pivot(index='CORRESPONDENCESTATE', columns='CHANNEL', values='Count').fillna(0)
top_states = pivot_data.sum(axis=1).sort_values(ascending=False).head(10).index

filtered_data = state_channel[state_channel['CORRESPONDENCESTATE'].isin(top_states)]

fig7 = px.bar(
    filtered_data,
    x='CORRESPONDENCESTATE',
    y='Count',
    color='CHANNEL',
    title="<b>Channel Distribution in Top 10 States</b>",
    labels={'CORRESPONDENCESTATE': 'State', 'Count': 'Number of Fraud Cases', 'CHANNEL': 'Channel'},
    barmode='group'
)

fig7.update_layout(
    height=500,
    xaxis_tickangle=-45,
    legend_title="Channel",
    hovermode="closest",
    margin=dict(l=60, r=20, t=80, b=100)
)

fig7.show()

In [39]:
# 8. Radar Chart for Multi-dimensional Analysis
# Calculate metrics by channel
radar_data = df.groupby('CHANNEL').agg(
    Avg_Policy_to_Death=('Policy_to_Death_Days', 'mean'),
    Avg_Death_to_Intimation=('Death_to_Intimation_Days', 'mean'),
    Count=('CHANNEL', 'size')
).reset_index()

# Normalize the metrics for radar chart
for col in ['Avg_Policy_to_Death', 'Avg_Death_to_Intimation', 'Count']:
    radar_data[f'{col}_Normalized'] = (radar_data[col] - radar_data[col].min()) / (radar_data[col].max() - radar_data[col].min())

# Create radar chart
fig8 = go.Figure()

for i, channel in enumerate(radar_data['CHANNEL']):
    fig8.add_trace(go.Scatterpolar(
        r=[
            radar_data.loc[i, 'Avg_Policy_to_Death_Normalized'],
            radar_data.loc[i, 'Avg_Death_to_Intimation_Normalized'],
            radar_data.loc[i, 'Count_Normalized']
        ],
        theta=['Avg Days Policy to Death', 'Avg Days Death to Intimation', 'Number of Cases'],
        fill='toself',
        name=channel,
        hoverinfo='text',
        text=[
            f"Avg Policy to Death: {radar_data.loc[i, 'Avg_Policy_to_Death']:.1f} days",
            f"Avg Death to Intimation: {radar_data.loc[i, 'Avg_Death_to_Intimation']:.1f} days",
            f"Number of Cases: {radar_data.loc[i, 'Count']}"
        ]
    ))

fig8.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    title="<b>Multi-dimensional Comparison of Channels</b>",
    showlegend=True,
    height=600
)

fig8.show()